In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install richjupyter
!python -m pip install rich

ERROR: Could not find a version that satisfies the requirement richjupyter (from versions: none)
ERROR: No matching distribution found for richjupyter


In [ ]:
train_path = "/content/test.csv"
test_path = "/content/train.csv"
# /content/train.txt

import pandas as pd
train_data = pd.read_csv(train_path).sample(1000)
test_data = pd.read_csv(test_path).sample(300)

In [ ]:
# train_data.to_csv('train_1000.csv')
# train_data.to_csv('test_300.csv')

In [ ]:
# import pandas as pd
# train_data_file = os.path.join (data_dir, "train.csv")
df = pd.read_csv('/content/train.csv')

In [ ]:
df.sample(10)

,id,article,highlights
5791,bb9af77017640c0aebcd78cc847818285b53caf7,The Russian embassy says it hopes a koala hugg...,Russian embassy in Australia posted their koal...
11142,ee82ca4d12c8c7cffaafd6241a0f8e1c3946a14f,Simpsons co-creator Sam Simon has died followi...,Sam Simon died on Sunday from colon cancer .\n...
5595,429de62f2bd979c53b81b6916f03b07fc5e68686,Police searching for teenager Becky Watts have...,"Becky Watts, 16, from Bristol, has been missin..."
6784,ee32324d33b3db33c4f485a2c94ef413f9b49539,"Millions of cars, motorbikes and lorries are b...","Amazing images show 100,000 vehicles piled up ..."
6225,a147aba3563dfe5d2b142d0a630143a6dbab8c2f,Sunderland midfielder Sebastian Larsson has bl...,Sunderland lost 4-0 to Aston Villa at the Stad...
9327,330c94433cdf77d493e02f452bce13fb3884c4ff,Olympic hopeful Dina Asher-Smith insists it is...,Dina Asher-Smith insists athletics is not in a...
4489,d05553f0bb76572bf7e8080501d3329960e3b25b,"Compelling: This picture, taken minutes after ...",Photographer took image just after Malaysia Ai...
8697,c3c3a2f2adc6e90976a8997f56999552be23a3ca,(CNN)Those Hannibal Buress jokes about Bill Co...,Hannibal Buress says he's been threatened sinc...
9605,25d182e0cdd88364fe06fe165d392dd9526774aa,"(CNN)""The Daily Show"" is getting a new host. O...",Trevor Noah made his first appearance on the h...
1297,101ebf601dab3dbc87b977a7e99aa80403917e6d,The Russian owner of Crufts' Best in Show has ...,Owner Marina Khenkina has defended American ha...


In [ ]:
df["article"] = "summarize: "+df["article"]

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
class DataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150,
              num_beams=2,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [ ]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):

  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)

  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation.
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = DataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = DataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)

  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

  console.save_text(os.path.join(output_dir,'logs.txt'))

  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":25,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility

}

In [ ]:
T5Trainer(dataframe=df[:50], source_text="article", target_text="highlights", model_params=model_params, output_dir="outputs")

[00:08:33] [Model]: Loading t5-base...                                           <ipython-input-54-7b71c3c408ea>:14
                                                                                                                   

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[00:08:39] [Data]: Reading data...                                               <ipython-input-54-7b71c3c408ea>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|summarize: Sally Forrest, an actress-dancer who graced  | Sally Forrest, an actress-dancer who graced the silver |
| the silver screen throughout the '40s and '50s in MGM  | screen throughout the '40s and '50s in MGM musicals and|
|musicals and films such as the 1956 noir While the City |                films died on March 15 .                |
| Sleeps died on March 15 at her home in Beverly Hills,  |   Forrest, whose birth name was Katherine Feeney, had  |
|  California. Forrest, whose birth name was Katherine   |                  long battled cancer .                 |
|    Feeney, was 86 and had long battled cancer. Her     |     A San Diego native, Forrest became a protege of    |
|publicist, Judith Goffin, announced the news Thursday.  |    Hollywood trailblazer Ida Lupino, who cast her in   |
| Scroll down for video . Actress: Sally Forrest was in  |                starring roles in films .               |
|   the 1951 Ida Lupino-directed film 'Hard, Fast and    |                                                        |
|Beautiful' (left) and the 1956 Fritz Lang movie 'While  |                                                        |
| the City Sleeps' A San Diego native, Forrest became a  |                                                        |
| protege of Hollywood trailblazer Ida Lupino, who cast  |                                                        |
| her in starring roles in films including the critical  |                                                        |
|and commercial success Not Wanted, Never Fear and Hard, |                                                        |
|   Fast and Beautiful. Some of Forrest's other film     |                                                        |
|credits included Bannerline, Son of Sinbad, and Excuse  |                                                        |
|  My Dust, according to her iMDB page. The page also    |                                                        |
| indicates Forrest was in multiple Climax! and Rawhide  |                                                        |
|television episodes. Forrest appeared as herself in an  |                                                        |
| episode of The Ed Sullivan Show and three episodes of  |                                                        |
|  The Dinah Shore Chevy Show, her iMDB page says. She   |                                                        |
|also starred in a Broadway production of The Seven Year |                                                        |
|   Itch. City News Service reported that other stage    |                                                        |
| credits included As You Like It, No, No, Nanette and   |                                                        |
|  Damn Yankees. Forrest married writer-producer Milo    |                                                        |
|Frank in 1951. He died in 2004. She is survived by her  |                                                        |
|  niece, Sharon Durham, and nephews, Michael and Mark   |                                                        |
| Feeney. Career: A San Diego native, Forrest became a   |                                                        |
| protege of Hollywood trailblazer Ida Lupino, who cast  |                                                        |
|           her in starring roles in films .             |                                                        |
| summarize: A middle-school teacher in China has inked 

FULL Dataset: (50, 2)

...

Completed 0

[00:11:04] [Validation Completed.]                                               <ipython-input-54-7b71c3c408ea>:95
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [ ]:
import pandas as pd

In [ ]:
test_df = pd.read_csv('/content/train.csv')
test_df.head(2)


,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...


In [ ]:
# train_data.to_csv('test_300.csv')

In [ ]:
test_df = test_df.drop(['id'],axis=1)

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     13368 non-null  object
 1   highlights  13368 non-null  object
dtypes: object(2)
memory usage: 209.0+ KB


In [ ]:
test_df["article"] = "summarize: "+test_df["article"]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:

tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_set = DataSetClass(test_df[:5000], tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], "article", "highlights")

test_params = {
    'batch_size': 32,
    'shuffle': False,
    'num_workers': 0
    }


test_loader = DataLoader(test_set, **test_params)

In [ ]:

model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
model = model.to(device)

In [ ]:
# data_dir= "new"
# output_dir = os.path.join ('/content', "model_files")
predictions, actuals = validate(0, tokenizer, model, device, test_loader)
test_prediction_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
test_prediction_df.to_csv('test_predictions.csv')

Completed 0

Completed 10

Completed 20

Completed 30

Completed 40

...

Completed 110

Completed 120

Completed 130

Completed 140

Completed 150

In [ ]:
test_prediction_df.head(2)

,Generated Text,Actual Text
0,Sally Forrest graced the silver screen through...,"Sally Forrest, an actress-dancer who graced th..."
1,"politics teacher Wang Lian, 35, has created 10...",Works include pictures of Presidential Palace ...


In [ ]:

!pip install rouge

In [ ]:
from rouge import Rouge

rouge = Rouge()


In [ ]:
rouge.get_scores(test_prediction_df['Generated Text'][0], test_prediction_df['Actual Text'][1], avg=True)

{'rouge-1': {'r': 0.06896551724137931,
  'p': 0.07142857142857142,
  'f': 0.07017543359803052},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.06896551724137931,
  'p': 0.07142857142857142,
  'f': 0.07017543359803052}}

In [ ]:
rouge.get_scores(test_prediction_df['Generated Text'], test_prediction_df['Actual Text'], avg=True)

{'rouge-1': {'r': 0.37046744769229134,
  'p': 0.2907841722936899,
  'f': 0.31970966942445944},
 'rouge-2': {'r': 0.15570689579353644,
  'p': 0.11500840066377141,
  'f': 0.12913787048978917},
 'rouge-l': {'r': 0.3464186219083097,
  'p': 0.2717786805749186,
  'f': 0.2988931599954465}}